In [25]:
import pandas as pd

csv_file = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wine_raw = pd.read_csv(csv_file, sep=";")
wine_data = wine_raw
wine_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [26]:
import random
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import neighbors
from sklearn import datasets
from sklearn import metrics
import math

# Definir seed utilizada
random.seed(1001001)

# Definir as features
all_features = ['fixed acidity', 'volatile acidity', 'citric acid', 
            'residual sugar', 'chlorides', 'free sulfur dioxide', 
            'total sulfur dioxide', 'density', 'pH', 
            'sulphates', 'alcohol', 'quality']
features = ['fixed acidity', 'volatile acidity', 'citric acid', 
            'residual sugar', 'chlorides', 'free sulfur dioxide', 
            'total sulfur dioxide', 'density', 'pH', 
            'sulphates', 'alcohol']
outcome_column = 'quality'

outcome_labels = sorted(list(set(wine_data.quality)))
print(outcome_labels)

X = np.array(wine_data[features])
Y = np.array(wine_data[outcome_column])

# Separar os conjuntos de treino e teste
Wine_Train, Wine_Test = train_test_split(wine_data, test_size=0.2, stratify=wine_data[outcome_column])

# Converter os conjuntos de treino para array
X_train = np.array(Wine_Train[features])
Y_train = np.array(Wine_Train[outcome_column])
print(X_train.shape[0], Y_train.shape[0])

# Converter os conjuntos de teste para array
X_test = np.array(Wine_Test[features])
Y_test = np.array(Wine_Test[outcome_column])
print(X_test.shape[0], Y_test.shape[0])

[3, 4, 5, 6, 7, 8]
1279 1279
320 320


In [132]:
from sklearn import metrics

#Create an instance of K-nearest neighbor classifier
knn_model = neighbors.KNeighborsClassifier(n_neighbors=5)

#Train the classifier
knn_model.fit(X_train,Y_train)

Yhat_train = knn_model.predict(X_train)

#Compute the prediction according to the model
Yhat = knn_model.predict(X_test)
print(Yhat[4])
print ('Predicted value: ' + str(Yhat[-1]), ', real target: ' + str(Y[-1]))

_Precision = 1
_Recall = 2
_F1Score = 3
_Accuracy = 4

_AverageOfAllScores = 1 # Calcula a precisão/f1_score/recall de cada label e depois faz a média delas
_OverallScore = 2 # Calcula a precisão/f1_score/recall global
_ScoreByLabel = 3 # Calcula a precisão/f1_score/recall de cada label e retorna elas em um array

def calculateScores(real_group, pred_group, score_type, average_type):
    if(average_type == _AverageOfAllScores):
        chosenAverage = "macro"
    elif(average_type == _OverallScore):
        chosenAverage = "micro"
    elif(average_type == _ScoreByLabel):
        chosenAverage = None
    
    if(score_type == _Precision):
        return metrics.precision_score(y_true=real_group, y_pred=pred_group, average=chosenAverage)
    elif(score_type == _Recall):
        return metrics.recall_score(y_true=real_group, y_pred=pred_group, average=chosenAverage)
    elif(score_type == _F1Score):
        return metrics.f1_score(y_true=real_group, y_pred=pred_group, average=chosenAverage)
    elif(score_type == _Accuracy):
        # Normalizar os dados: Calcula a acurácia
        # Sem normalizar os dados: Calcula o número total de TP
        return metrics.accuracy_score(y_true=real_group, y_pred=pred_group, normalize=True)
    
    
scores = ['Precision', 'Recall', 'F1 Score', 'Accuracy']

def showScoreResults(score_name, real_group, pred_group):
    if(score_name == scores[0]):
        method = _Precision
    elif(score_name == scores[1]):
        method = _Recall
    elif(score_name == scores[2]):
        method = _F1Score
    elif(score_name == scores[3]):
        method = _Accuracy
        
    if(method == _Precision or method == _Recall or method == _F1Score):
        score = calculateScores(real_group, pred_group, method, _AverageOfAllScores)
        print('Average of all labels' + score_name + ':', '{:6.4f}'.format(score))
        score = calculateScores(real_group, pred_group, method, _OverallScore)
        print('Overall' + score_name + ':', '{:6.4f}'.format(score))
        score = calculateScores(real_group, pred_group, method, _ScoreByLabel)
        print(score_name + ' By Label')
        for label in outcome_labels:
            index = outcome_labels.index(label)
            print('Label: ' + str(label) + '\t' + '{:6.4f}'.format(score[index]))
    elif(method == _Accuracy):
        score = calculateScores(real_group, pred_group, method, 0)
        print(score_name + ':', '{:6.4f}'.format(score))

def showScores_TrainAndTest():
    for score in scores:
        print('\n')
        print('# ' + score)
        print('#')
        print('# Training Data')
        showScoreResults(score, Y_train, Yhat_train)
        print('#')
        print('# Test Data')
        showScoreResults(score, Y_test, Yhat)
        
showScores_TrainAndTest(); 


# f1_score = metrics.f1_score(y_true=Y_train, y_pred=Yhat_train, average="micro")
# print('f1_score in training data:', '{:6.4f}'.format(f1_score))
# f1_score = metrics.f1_score(y_true=Y_test, y_pred=Yhat, average="micro")
# print('f1_score in test data:', '{:6.4f}'.format(f1_score))

# ac = metrics.accuracy_score(y_true=Y_train, y_pred=Yhat_train)
# print('ac in training data:', '{:6.4f}'.format(ac))
# ac = metrics.accuracy_score(y_true=Y_test, y_pred=Yhat)
# print('ac in test data:', '{:6.4f}'.format(ac))

accuracy_train = knn_model.score(X_train, Y_train)
print('accuracy in training data:', '{:6.4f}'.format(accuracy_train))
accuracy_test = knn_model.score(X_test, Y_test)
print('accuracy in test data:    ', '{:6.4f}'.format(accuracy_test))

6
Predicted value: 5 , real target: 6


# Precision
#
# Training Data
Average of all labelsPrecision: 0.5863
OverallPrecision: 0.6677
Precision By Label
Label: 3	1.0000
Label: 4	0.5000
Label: 5	0.6672
Label: 6	0.6701
Label: 7	0.6807
Label: 8	0.0000
#
# Test Data
Average of all labelsPrecision: 0.2313
OverallPrecision: 0.5094
Precision By Label
Label: 3	0.0000
Label: 4	0.0000
Label: 5	0.5593
Label: 6	0.4754
Label: 7	0.3529
Label: 8	0.0000


# Recall
#
# Training Data
Average of all labelsRecall: 0.4118
OverallRecall: 0.6677
Recall By Label
Label: 3	0.3750
Label: 4	0.1429
Label: 5	0.7982
Label: 6	0.6451
Label: 7	0.5094
Label: 8	0.0000
#
# Test Data
Average of all labelsRecall: 0.2218
OverallRecall: 0.5094
Recall By Label
Label: 3	0.0000
Label: 4	0.0000
Label: 5	0.7279
Label: 6	0.4531
Label: 7	0.1500
Label: 8	0.0000


# F1 Score
#
# Training Data
Average of all labelsF1 Score: 0.4558
OverallF1 Score: 0.6677
F1 Score By Label
Label: 3	0.5455
Label: 4	0.2222
Label: 5	0.7268
Label: 6	0.6573


C:\Users\gabi_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\gabi_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [122]:
# Matriz de Confusão
confm = metrics.confusion_matrix(Y_test, Yhat, labels=outcome_labels)
confmT = confm.T

wine_ConfusionMatrix = pd.DataFrame(confmT)
wine_ConfusionMatrix.columns = ['true_' + str(val) for val in outcome_labels]
wine_ConfusionMatrix.index   = ['pred_' + str(val) for val in outcome_labels]
wine_ConfusionMatrix

,true_3,true_4,true_5,true_6,true_7,true_8
pred_3,0,0,0,0,0,0
pred_4,0,0,1,3,0,0
pred_5,1,6,99,62,8,1
pred_6,0,3,34,58,26,1
pred_7,1,2,2,5,6,1
pred_8,0,0,0,0,0,0


In [123]:
# Matriz de Confusão Normalizada
# sumPerActualQuality = wine_ConfusionMatrix.apply(sum, axis=0)
# print(sumPerActualQuality)
# dfWineCMNormalizedByActualValue = wine_ConfusionMatrix.apply((lambda x: x/sumPerActualQuality), axis=1)
# dfWineCMNormalizedByActualValue

In [124]:
# Definindo TP, TN, FP e FN para a matriz de confusão
# FIXME: Não sei se para a avaliação usamos a matriz normal gerada ou a normalizada, por enquanto estou usando a normal

TP = {}
for i in outcome_labels:
    predicted = 'pred_' + str(i)
    real = 'true_' + str(i)
    # Marcar como "true positive" os elementos que foram preditos como i e eram de fato i
    TP[i] = wine_ConfusionMatrix.loc[predicted, real]
    
FP = {}
for i in outcome_labels:
    predicted = 'pred_' + str(i)
    real = 'true_' + str(i)
    FP[i] = 0
    j = 0
    # Para os elementos que foram preditos como i
    for elements_value in wine_ConfusionMatrix.loc[predicted]:
        elements = 'true_' + str(outcome_labels[j])
        # Mas não forem de fato i, marcar como "false positive"
        if(elements != real):
            FP[i] += elements_value
        j += 1
        
FN = {}
for i in outcome_labels:
    predicted = 'pred_' + str(i)
    real = 'true_' + str(i)
    FN[i] = 0
    j = 0
    # Para os elementos que eram de fato i
    for elements_value in wine_ConfusionMatrix.loc[:,real]:
        elements = 'pred_' + str(outcome_labels[j])
        # Mas não foram preditos como i, marcar como "false negative"
        if(elements != predicted):
            FN[i] += elements_value
        j += 1

TN = {}
for i in outcome_labels:
    predicted = 'pred_' + str(i)
    real = 'true_' + str(i)
    TN[i] = 0
    # Para todos os elementos
    for index, row in wine_ConfusionMatrix.iterrows():
        # Que não foram preditos como i
        if(index != predicted):
            for j in outcome_labels:
                elements = 'true_' + str(j)
                # E que de fato não eram i, marcar como "true negative"
                if(elements != real):
                    elements_value = row[elements]
                    TN[i] += elements_value
                    
print('TP', TP)
print('FP', FP)
print('FN', FN)
print('TN', TN)

TP {3: 0, 4: 0, 5: 99, 6: 58, 7: 6, 8: 0}
FP {3: 0, 4: 4, 5: 78, 6: 64, 7: 11, 8: 0}
FN {3: 2, 4: 11, 5: 37, 6: 70, 7: 34, 8: 3}
TN {3: 318, 4: 305, 5: 106, 6: 128, 7: 269, 8: 317}


In [134]:
# Métricas de Qualidade

# Precision by label in outcome_labels
# precision = TP / (TP + FP)

print("# PRECISION")
for i in outcome_labels:
    precision = (TP[i] / (TP[i] + FP[i]))
    print('precision for wines of quality ' + str(i) + '   ', '{:7.4f}'.format(precision))

# print('precision   ', '{:7.4f}'.format(metrics.precision_score(Y_test, Yhat)), '(from sklearn.metrics)')

# Recall/Sensitivity by label in outcome_labels
# recall/sensitivity = TP / (TP + FN)

print("\n# RECALL / SENSITIVITY")
for i in outcome_labels:
    recall = (TP[i] / (TP[i] + FN[i]))
    print('recall for wines of quality ' + str(i) + '   ', '{:7.4f}'.format(recall))

# print('recall      ', '{:7.4f}'.format(metrics.recall_score(Y_test, Yhat)), '(from sklearn.metrics)')

# Accuracy
# accuracy = (TP + TN) / (TP + TN + FP + FN)

print("\n# ACCURACY")

def sum_dict_values (dict_values):
    sum_ = 0
    for i in outcome_labels:
        sum_ += dict_values[i]
    if(math.isnan(sum_)):
        return 0
    else:
        return sum_
    
sumTP = sum_dict_values(TP)
sumTN = sum_dict_values(TN)
sumFP = sum_dict_values(FP)
sumFN = sum_dict_values(FN)

# accuracy = ((sumTP + sumTN) / (sumTP + sumTN + sumFP + sumFN))
accuracy = ((sumTP ) / (sumFP + sumFN))
print('accuracy' + '   ', '{:7.4f}'.format(accuracy))
print(sumTP, sumTN)

print('accuracy    ', '{:7.4f}'.format(metrics.accuracy_score(Y_test, Yhat)), '(from sklearn.metrics)')

# F1 Score by label in outcome_labels
# F1_score = 2 * precision * recall / (precision + recall)

print("\n# F1 SCORE")
for i in outcome_labels:
    precision = (TP[i] / (TP[i] + FP[i]))
    recall = (TP[i] / (TP[i] + FN[i]))
    F1_score = ((2 * precision * recall) / (precision + recall))
    print('F1 Score for wines of quality ' + str(i) + '   ', '{:7.4f}'.format(F1_score))

# print('F1_score    ', '{:7.4f}'.format(metrics.f1_score(Y_test, Yhat)), '(from sklearn.metrics)')

# PRECISION
precision for wines of quality 3        nan
precision for wines of quality 4     0.0000
precision for wines of quality 5     0.5593
precision for wines of quality 6     0.4754
precision for wines of quality 7     0.3529
precision for wines of quality 8        nan

# RECALL / SENSITIVITY
recall for wines of quality 3     0.0000
recall for wines of quality 4     0.0000
recall for wines of quality 5     0.7279
recall for wines of quality 6     0.4531
recall for wines of quality 7     0.1500
recall for wines of quality 8     0.0000

# ACCURACY
accuracy     0.5191
163 1443
accuracy      0.5094 (from sklearn.metrics)

# F1 SCORE
F1 Score for wines of quality 3        nan
F1 Score for wines of quality 4        nan
F1 Score for wines of quality 5     0.6326
F1 Score for wines of quality 6     0.4640
F1 Score for wines of quality 7     0.2105
F1 Score for wines of quality 8        nan


C:\Users\gabi_\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\gabi_\Anaconda3\lib\site-packages\ipykernel\__main__.py:54: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\gabi_\Anaconda3\lib\site-packages\ipykernel\__main__.py:56: RuntimeWarning: invalid value encountered in double_scalars
